In [1]:
import Max3SAT
import solver
import utils

import random
import json
import numpy as np

In [2]:
testsets_literals_count = [5, 10, 15, 20, 50, 75, 100, 125]
testsets_clauses_count = [15, 30, 55, 91, 218, 325, 430, 538]
solvers_name = ["FixStars1s", "FixStars10s", "FixStars100s", "Gurobi10s", "Gurobi100s", "Gurobi150s", "DWave4.1", "DWave6.4", "DWaveV2"]

In [3]:
qubo_size = np.array(testsets_literals_count) + np.array(testsets_clauses_count)
print("QUBO size:", qubo_size)

QUBO size: [ 20  40  70 111 268 400 530 663]


In [4]:
print("literals difference:", np.diff(testsets_literals_count))
print("Clauses difference:", np.diff(testsets_clauses_count))

literals difference: [ 5  5  5 30 25 25 25]
Clauses difference: [ 15  25  36 127 107 105 108]


In [5]:
literals_count = 15
clauses_count = 55
clauses, num_n, num_m = utils.generator_3SAT(literals_count, clauses_count)
print("num_n:", num_n)
print("num_m:", num_m)
print("clauses:", clauses)

num_n: 15
num_m: 55
clauses: [[ -8  -6  11]
 [ -9  -2  13]
 [ -1   6   7]
 [  7  12  13]
 [ -7   1  12]
 [  3   4   6]
 [ -8  -2  12]
 [  6   7  11]
 [ -6  -1   8]
 [  8  11  15]
 [ -9   9  11]
 [-14 -13   8]
 [ -2   5  13]
 [-10   8  13]
 [ -9   9  13]
 [-12  -4   2]
 [ -6  -1   1]
 [-13   3   9]
 [ -9   8  11]
 [ -7  -2   3]
 [-13   2  13]
 [-12  -2   2]
 [-14  -9  11]
 [-10  -3  12]
 [-14   6   9]
 [ -4   4   9]
 [-15  -8  10]
 [-13   9  14]
 [-10  -4  15]
 [-12  -4  -3]
 [ -8   5  15]
 [ -5   1   7]
 [-13  -8  11]
 [ -8  -6  12]
 [-15 -14  15]
 [-15 -11   5]
 [  5  10  13]
 [ -8  -1  11]
 [ -2   5  14]
 [  5  12  13]
 [-11 -10  -9]
 [ -8  -2  15]
 [  2   8  11]
 [ -4   2  11]
 [ -4   7   8]
 [ -6   4   8]
 [-13   2   3]
 [  5  10  12]
 [-15   1  15]
 [ -8  -2  14]
 [ -8   8  10]
 [ -4  13  15]
 [ -3   8  10]
 [-14 -11  15]
 [-15  11  14]]


In [6]:
testsets_use = 8
iterations_num = 10
solvers_name = ["FixStars10s", "Gurobi10s", "DWave6.4"]

results = []

for testsets in range(testsets_use):
    # Use the same problem for all iterations
    problem_num = f"0{random.randint(1, 100)}"
    literals_count = testsets_literals_count[testsets]
    clauses_count = testsets_clauses_count[testsets]

     # Select random 3-SAT problem
    if (literals_count + clauses_count < 100):
        clauses, num_n, num_m = utils.generator_3SAT(literals_count, clauses_count)
    else:
        filename = f'TestSets\\Satisfiable\\uf{literals_count}-{clauses_count}\\uf{literals_count}-{problem_num}.cnf'
        clauses, num_n, num_m = utils.read_cnf_file(filename)

    for iteration in range(iterations_num):
        iteration_results = []  # Collect results for each iteration

        # Formulate the QUBO and Model
        problem = Max3SAT.Max3SAT(num_n, clauses)
        nonzero_elements_percentage = problem.nonzero_element_percentage
        qubo_formulation_time = problem.time_QUBO_formulation

        to_solve = solver.solver(problem)
        model_formulation_time = to_solve.time_model_formulation
            
        # Solve the problem with each solver
        for name in solvers_name:
            max_clauses_satisfied_Obj, max_clauses_satisfied_Verified, execution_time, total_time = to_solve.solve_with_client(name)

            iteration_results.append({
                "solver": name,
                "results": {
                    "max_clauses_satisfied_Obj": max_clauses_satisfied_Obj,
                    "max_clauses_satisfied_Verified": max_clauses_satisfied_Verified,
                    "execution_time": execution_time,
                    "total_time": total_time
                }
            })
    
        # Bundle results for this iteration
        results.append({
            "parameters": {
                        "literals_count": literals_count,
                        "clauses_count": clauses_count,
                        "iteration": iteration + 1,
                        "problem_num": problem_num,
                        "qubo_formulation_time": qubo_formulation_time,
                        "model_formulation_time": model_formulation_time
            },
            "iteration_results": iteration_results
        })

# Convert everything else to string
def convert_everything_to_strings(obj):
    if isinstance(obj, (dict, list)):
        if isinstance(obj, dict):
            return {str(k): convert_everything_to_strings(v) for k, v in obj.items()}
        return [convert_everything_to_strings(x) for x in obj]
    return str(obj)  

serializable_results = convert_everything_to_strings(results)

# Save the results to a JSON file
output_file = "results.json"
with open(output_file, "w") as json_file:
    json.dump(serializable_results, json_file, indent=4)

print(f"Results saved to {output_file}")

Finished QUBO formulation. [5, 15]
QUBO formulation time: 0.8230276107788086
Finished model formulation.
Model formulation time: 0.0009887218475341797
FixStars10s Successfully solve, Total time: 10.799092054367065
Gurobi10s Successfully solve, Total time: 0.014002561569213867
DWave6.4 Successfully solve, Total time: 4.43706750869751
Finished QUBO formulation. [5, 15]
QUBO formulation time: 0.380448579788208
Finished model formulation.
Model formulation time: 0.0
FixStars10s Successfully solve, Total time: 10.863237380981445
Gurobi10s Successfully solve, Total time: 0.01601409912109375
DWave6.4 Successfully solve, Total time: 3.351423501968384
Finished QUBO formulation. [5, 15]
QUBO formulation time: 0.458965539932251
Finished model formulation.
Model formulation time: 0.0
FixStars10s Successfully solve, Total time: 10.856259822845459
Gurobi10s Successfully solve, Total time: 0.016002655029296875
DWave6.4 Successfully solve, Total time: 3.454871654510498
Finished QUBO formulation. [5, 1

2025-01-18 23:35:18 [amplify:WARNING] Connection timeout, retrying after sleep 1 second (1/3)


FixStars10s Successfully solve, Total time: 12.680684804916382
Gurobi10s Successfully solve, Total time: 10.044946908950806
DWave6.4 Successfully solve, Total time: 17.316855430603027
Finished QUBO formulation. [75, 325]
QUBO formulation time: 24.737107276916504
Finished model formulation.
Model formulation time: 0.07886075973510742
FixStars10s Successfully solve, Total time: 10.57672905921936
Gurobi10s Successfully solve, Total time: 10.078177213668823
DWave6.4 Successfully solve, Total time: 23.12093734741211
Finished QUBO formulation. [75, 325]
QUBO formulation time: 22.720635414123535
Finished model formulation.
Model formulation time: 0.08300018310546875
FixStars10s Successfully solve, Total time: 10.589583158493042
Gurobi10s Successfully solve, Total time: 10.072475910186768
DWave6.4 Successfully solve, Total time: 21.37829327583313
Finished QUBO formulation. [75, 325]
QUBO formulation time: 23.38158416748047
Finished model formulation.
Model formulation time: 0.08203291893005371

In [7]:
testsets_use = 1
iterations_num = 10
solvers_name = ["DWave6.4"]

results = []

for testsets in range(testsets_use):
    # Use the same problem for all iteration

    for iteration in range(iterations_num):
        problem_num = f"0{random.randint(1, 100)}"
        literals_count = 100
        clauses_count = 430

        # Select random 3-SAT problem
        if (literals_count + clauses_count < 100):
            clauses, num_n, num_m = utils.generator_3SAT(literals_count, clauses_count)
        else:
            filename = f'TestSets\\Satisfiable\\uf{literals_count}-{clauses_count}\\uf{literals_count}-{problem_num}.cnf'
            clauses, num_n, num_m = utils.read_cnf_file(filename)
        iteration_results = []  # Collect results for each iteration

        # Formulate the QUBO and Model
        problem = Max3SAT.Max3SAT(num_n, clauses)
        nonzero_elements_percentage = problem.nonzero_element_percentage
        qubo_formulation_time = problem.time_QUBO_formulation

        to_solve = solver.solver(problem)
        model_formulation_time = to_solve.time_model_formulation
            
        # Solve the problem with each solver
        for name in solvers_name:
            max_clauses_satisfied_Obj, max_clauses_satisfied_Verified, execution_time, total_time = to_solve.solve_with_client(name)

            iteration_results.append({
                "solver": name,
                "results": {
                    "max_clauses_satisfied_Obj": max_clauses_satisfied_Obj,
                    "max_clauses_satisfied_Verified": max_clauses_satisfied_Verified,
                    "execution_time": execution_time,
                    "total_time": total_time
                }
            })
    
        # Bundle results for this iteration
        results.append({
            "parameters": {
                        "literals_count": literals_count,
                        "clauses_count": clauses_count,
                        "iteration": iteration + 1,
                        "problem_num": problem_num,
                        "qubo_formulation_time": qubo_formulation_time,
                        "model_formulation_time": model_formulation_time
            },
            "iteration_results": iteration_results
        })

# Convert everything else to string
def convert_everything_to_strings(obj):
    if isinstance(obj, (dict, list)):
        if isinstance(obj, dict):
            return {str(k): convert_everything_to_strings(v) for k, v in obj.items()}
        return [convert_everything_to_strings(x) for x in obj]
    return str(obj)  

serializable_results = convert_everything_to_strings(results)

# Save the results to a JSON file
output_file = "results.json"
with open(output_file, "w") as json_file:
    json.dump(serializable_results, json_file, indent=4)

print(f"Results saved to {output_file}")

Finished QUBO formulation. [100, 430]
QUBO formulation time: 40.34913611412048
Finished model formulation.
Model formulation time: 0.14800071716308594
DWave6.4 Successfully solve, Total time: 29.20426845550537
Finished QUBO formulation. [100, 430]
QUBO formulation time: 40.72577261924744
Finished model formulation.
Model formulation time: 0.1560807228088379
DWave6.4 Successfully solve, Total time: 27.965044736862183
Finished QUBO formulation. [100, 430]
QUBO formulation time: 36.801344871520996
Finished model formulation.
Model formulation time: 0.14896345138549805
Error during solving with DWave6.4: both clique and minor embeddings failed: the problem size may be too large to embed in the target graph
Finished QUBO formulation. [100, 430]
QUBO formulation time: 39.516265630722046
Finished model formulation.
Model formulation time: 0.15199828147888184
Error during solving with DWave6.4: both clique and minor embeddings failed: the problem size may be too large to embed in the target gr